# Happy Farm Planting Schedule

View and calculate planting schedule with date dependencies.

In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Display options for better table formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Load schedule
df = pd.read_csv('../data/schedules/planting_schedule.csv')

# Convert dates
df['target_date'] = pd.to_datetime(df['target_date'])
df['prep_start_date'] = pd.to_datetime(df['prep_start_date'], errors='coerce')

# Calculate prep start dates from duration
mask = df['prep_duration_weeks'].notna()
df.loc[mask, 'calculated_prep_start'] = df.loc[mask].apply(
    lambda row: row['target_date'] - timedelta(weeks=row['prep_duration_weeks']),
    axis=1
)

# Use calculated date if no manual date provided
df['effective_prep_start'] = df['prep_start_date'].fillna(df['calculated_prep_start'])

df

,task_type,crop,target_date,prep_method,prep_duration_weeks,prep_start_date,notes,calculated_prep_start,effective_prep_start
0,cover_crop_termination,Row 1-6 (for tomatoes),2026-02-25,silage_tarp,6.0,2026-01-14,Opaque silage tarp; 5-6 week termination,2026-01-14,2026-01-14
1,cover_crop_termination,Late-seeded rows,2026-02-25,clear_plastic,1.0,2026-02-18,Fast termination (days); warms soil for tomatoes,2026-02-18,2026-02-18
2,transplant,Tomatoes,2026-02-25,NaN,NaN,NaT,Transplant into prepared rows,NaT,NaT


## Full Schedule View

In [3]:
# Create display DataFrame
display_df = df[[
    'task_type', 'crop', 
    'effective_prep_start', 'prep_method', 'prep_duration_weeks',
    'target_date', 'notes'
]].copy()

# Format dates nicely
display_df['effective_prep_start'] = display_df['effective_prep_start'].dt.strftime('%Y-%m-%d (%a)')
display_df['target_date'] = display_df['target_date'].dt.strftime('%Y-%m-%d (%a)')

# Rename for clarity
display_df = display_df.rename(columns={
    'effective_prep_start': 'start_prep',
    'prep_duration_weeks': 'weeks'
})

display_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

task_type,crop,start_prep,prep_method,weeks,target_date,notes
cover_crop_termination,Row 1-6 (for tomatoes),2026-01-14 (Wed),silage_tarp,6.000000,2026-02-25 (Wed),Opaque silage tarp; 5-6 week termination
cover_crop_termination,Late-seeded rows,2026-02-18 (Wed),clear_plastic,1.000000,2026-02-25 (Wed),Fast termination (days); warms soil for tomatoes
transplant,Tomatoes,nan,nan,nan,2026-02-25 (Wed),Transplant into prepared rows


## Upcoming Tasks (Next 60 Days)

In [4]:
today = pd.Timestamp.now()
upcoming = df[
    (df['effective_prep_start'] >= today) &
    (df['effective_prep_start'] <= today + timedelta(days=60))
].sort_values('effective_prep_start').copy()

if not upcoming.empty:
    # Calculate days until
    upcoming['days_until'] = (upcoming['effective_prep_start'] - today).dt.days
    
    upcoming_display = upcoming[[
        'days_until', 'effective_prep_start', 'task_type', 'crop',
        'prep_method', 'target_date', 'notes'
    ]].copy()
    
    upcoming_display['effective_prep_start'] = upcoming_display['effective_prep_start'].dt.strftime('%Y-%m-%d (%a)')
    upcoming_display['target_date'] = upcoming_display['target_date'].dt.strftime('%Y-%m-%d (%a)')
    
    upcoming_display.style.set_properties(**{'text-align': 'left'}).hide(axis='index')
else:
    print("No upcoming tasks in the next 60 days")

## Timeline Visualization (Optional)

In [ ]:
# Uncomment if you want a simple timeline chart
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates

# fig, ax = plt.subplots(figsize=(12, 6))

# for idx, row in df.iterrows():
#     ax.barh(idx, 
#             (row['target_date'] - row['effective_prep_start']).days,
#             left=row['effective_prep_start'],
#             height=0.5,
#             label=row['crop'])
#     ax.text(row['effective_prep_start'], idx, f" {row['crop']}", 
#             va='center', fontsize=9)

# ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
# ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
# plt.xticks(rotation=45)
# ax.set_yticks([])
# ax.set_xlabel('Date')
# ax.set_title('Planting Schedule Timeline')
# plt.tight_layout()
# plt.show()